# KITTI Dataset

In [ ]:
import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm

## Data Preprocessing - SGM disparity maps

In [8]:
def center_crop_to_square(img):
    h, w = img.shape
    crop_size = min(h, w)
    start_x = (w - crop_size) // 2
    start_y = (h - crop_size) // 2
    return img[start_y:start_y + crop_size, start_x:start_x + crop_size]

def generate_sgm_disparity_map(left_img_path, right_img_path, output_size=(64, 64), num_disparities=64):
    left_img = cv2.imread(str(left_img_path), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_img_path), cv2.IMREAD_GRAYSCALE)

    window_size = 5
    min_disp = 0
    num_disp = (num_disparities // 16) * 16
    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=5,
        P1=8 * 1 * window_size ** 2,
        P2=32 * 1 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    disparity = stereo.compute(left_img, right_img).astype(np.float32) / 16.0
    disparity[disparity < min_disp] = -1

    disparity = center_crop_to_square(disparity)
    disparity_resized = cv2.resize(disparity, output_size, interpolation=cv2.INTER_NEAREST)

    return disparity_resized

def process_stereo_sequence(left_dir, right_dir, output_dir, output_size=(64, 64)):
    left_dir = Path(left_dir)
    right_dir = Path(right_dir)
    output_depth_dir = Path(output_dir) / "depths"
    output_image_dir = Path(output_dir) / "images"
    output_depth_dir.mkdir(parents=True, exist_ok=True)
    output_image_dir.mkdir(parents=True, exist_ok=True)

    left_images = sorted(left_dir.glob("*.png"))
    right_images = sorted(right_dir.glob("*.png"))

    for l_path, r_path in tqdm(zip(left_images, right_images), total=len(left_images), desc=f"Processing {left_dir.parent.parent.name}"):
        if l_path.stem != r_path.stem:
            print(f"Skipping mismatched pair: {l_path.name} vs {r_path.name}")
            continue

        try:
            disparity = generate_sgm_disparity_map(l_path, r_path, output_size)
            np.save(output_depth_dir / f"{l_path.stem}.npy", disparity)

            left_img = cv2.imread(str(l_path), cv2.IMREAD_GRAYSCALE)
            left_img = center_crop_to_square(left_img)
            left_resized = cv2.resize(left_img, output_size, interpolation=cv2.INTER_AREA)
            cv2.imwrite(str(output_image_dir / f"{l_path.stem}.png"), left_resized)

        except Exception as e:
            print(f"Error processing {l_path.name}: {e}")

In [ ]:
BASE_DIR = Path(r"") # Change this to be a path to your raw KITTI data. You will need the calibrated data.
OUTPUT_BASE = Path(r"SGM_Full") # This is the name of the folder where your preprocessed images and disparity maps will be saved.
for recording_dir in BASE_DIR.iterdir():
    if not recording_dir.is_dir():
        continue

    left_path = recording_dir / "image_02" / "data"
    right_path = recording_dir / "image_03" / "data"

    if left_path.exists() and right_path.exists():
        recording_name = recording_dir.name
        output_dir = OUTPUT_BASE / recording_name
        process_stereo_sequence(left_path, right_path, output_dir, output_size=(64, 64))
    else:
        print(f"Skipping {recording_dir.name}, missing image_02 or image_03 directories.")